In [1]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from torchmetrics import Accuracy, F1Score, Recall, Precision, ConfusionMatrix
import requests
from pathlib import Path
from timeit import default_timer as timer
from tqdm.auto import tqdm
print(torch.__version__)
print(torchvision.__version__)

2.8.0
0.23.0


/Users/alt/Library/CloudStorage/OneDrive-Personal/Portfolio/skill-builder-process/ml_basics/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cpu"

In [4]:
#Training func
def train_step(model:torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device = device,
               task="multiclass",
               num_classes=None) -> tuple:
    """Performs a training with model trying to learn on data_loader"""
    #Creating training and test loop
    model.train()
    acc_metric = Accuracy(task=task, num_classes=num_classes).to(device)
    total_loss, total_n  = 0, 0
    for X, y in tqdm(data_loader):
        X, y = X.to(device), y.to(device)
        #1. Forward pass
        logits = model(X)

        #2. Calculate loss (per batch)
        batch_loss = loss_fn(logits, y)

        #Calc total loss and n(number of sample(batch))
        n = y.size(0)
        total_loss += batch_loss.item() * n # accumulate train loss
        total_n += n

        #acc
        acc_metric.update(logits, y)

        #3 three steps
        optimizer.zero_grad(set_to_none=True); batch_loss.backward(); optimizer.step()



    # Scale loss and acc to find the average loss/acc per batch
    avg_loss = total_loss / total_n
    avg_acc = acc_metric.compute().item()
    print(f"Train loss: {avg_loss:.5f} | Train accuracy: {avg_acc * 100:.2f}%")
    return avg_loss, avg_acc


In [5]:
#Testing func
def test_step(model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              device: torch.device = 'cpu',
              task="multiclass",
              num_classes=None) -> tuple:
    """Returns a tuple that contains avg_loss, avg_acc"""
    acc_metric = Accuracy(task=task, num_classes=num_classes).to(device)

    total_loss = 0.0
    total_n = 0

    model.eval()
    with torch.inference_mode():
        for X, y in tqdm(data_loader):
            X, y = X.to(device), y.to(device)

            # Make predictions
            logits = model(X)
            batch_loss = loss_fn(logits, y)

            n = y.size(0)
            total_loss += batch_loss.item() * n
            total_n += n

            acc_metric.update(logits, y)
    # Scale loss and acc to find the average loss/acc per batch
    avg_loss = total_loss / total_n
    avg_acc = acc_metric.compute().item()
    print(f"Test loss: {avg_loss:.5f} | Test accuracy: {avg_acc * 100:.2f}%")
    return avg_loss, avg_acc

### Model 2: Builing a Convolutional NN 

In [7]:
class FashionMNISTModelV2(nn.Module):
    """
    Model architecture that replicates the TinyVGG
    model from CNN explainer website
    """
    def __init__(self, input_shade:int,
                 hidden_units: int,
                 output_shape: int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shade,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units)
        )

    def forward(self, x):
        x = self.conv_block_1(x)
        print(x.shape)
        x = self.conv_block_2(x),
        print(x.shape)
        x = self.classifier(x)
        return x

In [8]:
model_2  = FashionMNISTModelV2(input_shade=1,
                               hidden_units=10,
                               output_shape=1).to(device)